In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Amalia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Amalia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in laun

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [2]:
class AttentionBlock(nn.Module):
    """Módulo de Atención para filtrar características irrelevantes"""
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

class CloudAttentionUNet(nn.Module):
    def __init__(self):
        super(CloudAttentionUNet, self).__init__()
        
        base_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.encoder0 = nn.Sequential(*list(base_model.children())[:3])
        self.encoder1 = base_model.layer1
        self.encoder2 = base_model.layer2
        self.encoder3 = base_model.layer3
        self.encoder4 = base_model.layer4

        self.bottleneck = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(inplace=True)
        )

        self.att4 = AttentionBlock(F_g=512, F_l=512, F_int=256)
        self.att3 = AttentionBlock(F_g=256, F_l=256, F_int=128)
        self.att2 = AttentionBlock(F_g=128, F_l=128, F_int=64)
        self.att1 = AttentionBlock(F_g=64, F_l=64, F_int=32)

        self.up4 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.up3 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.up1 = nn.ConvTranspose2d(64, 64, 2, stride=2)
        
        self.out_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        e0 = self.encoder0(x)
        e1 = self.encoder1(e0)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)

        b = self.bottleneck(e4)

        d4 = self.up4(b)
        x4 = self.att4(g=d4, x=e3)
        
        d3 = self.up3(d4)
        x3 = self.att3(g=d3, x=e2)
        
        d2 = self.up2(d3)
        x2 = self.att2(g=d2, x=e1)
        
        d1 = self.up1(d2)
        x1 = self.att1(g=d1, x=e0)
        
        out = self.out_conv(d1)
        out = F.interpolate(out, size=x.shape[2:], mode='bilinear', align_corners=True)
        return torch.sigmoid(out)

In [3]:
class CloudTestDataset(Dataset):
    def __init__(self, images_dir, masks_dir, size=(256,256)):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.size = size

        self.files = sorted([
            f for f in os.listdir(images_dir)
            if f.endswith(('.png','.jpg','.jpeg','.tif'))
        ])

        self.img_tf = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485,0.456,0.406],
                std=[0.229,0.224,0.225]
            )
        ])

        self.mask_tf = transforms.Compose([
            transforms.Resize(size, interpolation=transforms.InterpolationMode.NEAREST),
            transforms.ToTensor(),
            transforms.Lambda(lambda x: (x > 0.5).float())
        ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]

        image = Image.open(
            os.path.join(self.images_dir, fname)
        ).convert("RGB")

        mask = Image.open(
            os.path.join(self.masks_dir, fname)
        ).convert("L")

        return self.img_tf(image), self.mask_tf(mask), fname

In [4]:
# Verificar directorios
test_img_dir = "overall-mask_test"
test_mask_dir = "masked_test"

if not os.path.exists(test_img_dir):
    raise FileNotFoundError(f"❌ Directorio de imágenes no encontrado: {test_img_dir}")
if not os.path.exists(test_mask_dir):
    raise FileNotFoundError(f"❌ Directorio de máscaras no encontrado: {test_mask_dir}")

print(f"✓ Directorio de imágenes: {test_img_dir}")
print(f"✓ Directorio de máscaras: {test_mask_dir}")

test_dataset = CloudTestDataset(
    images_dir=test_img_dir,
    masks_dir=test_mask_dir,
    size=(256, 256)
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False
)

print(f"✓ Dataset cargado con {len(test_dataset)} imágenes")

NameError: name 'os' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

model_path = "cloud_segmentation_model.pth"

if not os.path.exists(model_path):
    raise FileNotFoundError(f"❌ Archivo de modelo no encontrado: {model_path}")

model = CloudAttentionUNet().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

print(f"✓ Modelo cargado desde: {model_path}")
print(f"✓ Modelo en modo evaluación")

In [ ]:
import numpy as np

def dice_score(pred, target):
    pred = pred.astype(bool)
    target = target.astype(bool)
    inter = np.logical_and(pred, target).sum()
    return 2*inter / (pred.sum() + target.sum() + 1e-8)

def iou_score(pred, target):
    pred = pred.astype(bool)
    target = target.astype(bool)
    inter = np.logical_and(pred, target).sum()
    union = np.logical_or(pred, target).sum()
    return inter / (union + 1e-8)

In [ ]:
all_dice = []
all_iou = []
results_summary = []

print(f"Evaluando {len(test_dataset)} imágenes...\n")

with torch.no_grad():
    for batch_idx, (images, masks, fname) in enumerate(test_loader):
        try:
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)

            # Asegurar que tenemos la dimensión correcta
            preds = outputs.squeeze() if outputs.shape[1] == 1 else outputs
            preds = torch.sigmoid(preds) if preds.max() > 1 else preds
            preds_np = preds.cpu().numpy()
            
            # Manejar dimensiones
            if preds_np.ndim == 3:
                preds_np = preds_np[0]
            
            masks_np = masks.squeeze().cpu().numpy()
            binary_pred = (preds_np > 0.5).astype("float32")

            # Métricas
            dice = dice_score(binary_pred, masks_np)
            iou = iou_score(binary_pred, masks_np)

            all_dice.append(dice)
            all_iou.append(iou)
            results_summary.append({
                'filename': fname[0],
                'dice': dice,
                'iou': iou
            })

            # ===== Mostrar resultados =====
            img_np = images.cpu().numpy()[0].transpose(1, 2, 0)

            # Desnormalizar para visualizar
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_np = (img_np * std + mean).clip(0, 1)

            fig, axes = plt.subplots(1, 3, figsize=(15, 4))

            axes[0].imshow(img_np)
            axes[0].set_title(f"Imagen\n{fname[0]}")
            axes[0].axis("off")

            axes[1].imshow(masks_np, cmap="gray")
            axes[1].set_title("Máscara Real")
            axes[1].axis("off")

            axes[2].imshow(binary_pred, cmap="gray")
            axes[2].set_title(f"Predicción\nDice: {dice:.4f} | IoU: {iou:.4f}")
            axes[2].axis("off")

            plt.tight_layout()
            plt.show()
            
            print(f"[{batch_idx+1}/{len(test_dataset)}] {fname[0]}: Dice={dice:.4f}, IoU={iou:.4f}")
            
        except Exception as e:
            print(f"❌ Error procesando {fname[0]}: {str(e)}")
            continue

print("\n" + "="*50)
print("RESULTADOS PROMEDIO")
print("="*50)
print(f"Dice promedio: {np.mean(all_dice):.4f} (±{np.std(all_dice):.4f})")
print(f"IoU promedio:  {np.mean(all_iou):.4f} (±{np.std(all_iou):.4f})")
print(f"Total de imágenes procesadas: {len(all_dice)}")
print("="*50)